In [1]:
import zipfile
import os

# Set path to your zip file and extract location
zip_path = '/content/drive/MyDrive/AML Assignment 5/chicken_duck_dataset.zip'
extract_dir = '/content/drive/MyDrive/AML Assignment 5/chicken_duck_dataset/'

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Zip file extracted to:", extract_dir)

Zip file extracted to: /content/drive/MyDrive/AML Assignment 5/chicken_duck_dataset/


In [1]:
!pip install torch torchvision matplotlib scikit-learn

In [18]:
import os
import glob
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets.folder import default_loader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import torchvision.datasets as datasets
from tqdm import tqdm

In [12]:
# Class names
class_names = ['chicken', 'duck']

# Path to dataset
base_path = '/content/drive/MyDrive/AML Assignment 5/chicken_duck_dataset/chicken_duck_dataset'

# DEFINE DATA TRANSFORMATIONS
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [19]:
dataset = datasets.ImageFolder(root="/content/drive/MyDrive/AML Assignment 5/chicken_duck_dataset/chicken_duck_dataset", transform=transform)

# Print class mappings (Chicken → 0, Duck → 1)
print(f"Class Mapping: {dataset.class_to_idx}")

# Train-Validation-Test Split (70% Train, 15% Validation, 15% Test)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}, Test samples: {len(test_data)}")


Class Mapping: {'chicken': 0, 'duck': 1}
Training samples: 1078, Validation samples: 231, Test samples: 231


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [21]:
model = models.resnet50(pretrained=True)

# Replace the last fully connected layer for 2 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Chicken and Duck

model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
# DEFINE LOSS FUNCTION & OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [23]:
from sklearn.metrics import classification_report

def evaluate_model(loader, model, phase='Test'):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print(f"\n📊 {phase} Classification Report:")
    print(classification_report(
        all_labels,
        all_preds,
        labels=[0, 1],                      # 👈 This is essential!
        target_names=class_names           # ["Chicken", "Duck"]
    ))


In [24]:
# Evaluate model before fine-tuning
evaluate_model(test_loader, model, phase='Test')


📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       0.36      0.93      0.52        72
        duck       0.88      0.24      0.38       159

    accuracy                           0.45       231
   macro avg       0.62      0.58      0.45       231
weighted avg       0.72      0.45      0.42       231



In [25]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        # Update tqdm progress bar
        progress_bar.set_postfix(loss=running_loss / (total // train_loader.batch_size))

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")
    evaluate_model(test_loader, model, phase='Test')

print("Training complete!")

Epoch 1/5: 100%|██████████| 34/34 [00:26<00:00,  1.29it/s, loss=0.343]


Epoch [1/5], Loss: 0.3331, Train Accuracy: 85.25%

📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       0.87      0.93      0.90        72
        duck       0.97      0.94      0.95       159

    accuracy                           0.94       231
   macro avg       0.92      0.93      0.93       231
weighted avg       0.94      0.94      0.94       231



Epoch 2/5: 100%|██████████| 34/34 [00:15<00:00,  2.15it/s, loss=0.143]


Epoch [2/5], Loss: 0.1390, Train Accuracy: 96.01%

📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       0.96      0.93      0.94        72
        duck       0.97      0.98      0.97       159

    accuracy                           0.97       231
   macro avg       0.96      0.96      0.96       231
weighted avg       0.97      0.97      0.97       231



Epoch 3/5: 100%|██████████| 34/34 [00:15<00:00,  2.15it/s, loss=0.128]


Epoch [3/5], Loss: 0.1247, Train Accuracy: 95.73%

📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       0.96      0.93      0.94        72
        duck       0.97      0.98      0.97       159

    accuracy                           0.97       231
   macro avg       0.96      0.96      0.96       231
weighted avg       0.97      0.97      0.97       231



Epoch 4/5: 100%|██████████| 34/34 [00:15<00:00,  2.19it/s, loss=0.119]


Epoch [4/5], Loss: 0.1152, Train Accuracy: 96.57%

📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       1.00      0.79      0.88        72
        duck       0.91      1.00      0.95       159

    accuracy                           0.94       231
   macro avg       0.96      0.90      0.92       231
weighted avg       0.94      0.94      0.93       231



Epoch 5/5: 100%|██████████| 34/34 [00:15<00:00,  2.17it/s, loss=0.111]


Epoch [5/5], Loss: 0.1078, Train Accuracy: 95.92%

📊 Test Classification Report:
              precision    recall  f1-score   support

     chicken       0.93      0.94      0.94        72
        duck       0.97      0.97      0.97       159

    accuracy                           0.96       231
   macro avg       0.95      0.96      0.95       231
weighted avg       0.96      0.96      0.96       231

Training complete!
